In [1]:
import nnlibrary.configs.hvac_mode_classifier as cfg
from nnlibrary.engines import Trainer

In [2]:
trainer = Trainer(cfg=cfg)


HDF5 Dataset Info for train.h5:
  Input chunks: (100, 48, 16) (chunk = 0.29 MB)
  Output chunks: (100, 3) (chunk = 0.00 MB)

  Loading entire dataset into memory from /home/ahojrup/GitLab/mpc_ahu_neural_network/data/14days_2025-09-09_2025-09-23/dataset/train.h5...
    Expected memory usage:
      Inputs:  (16694, 48, 16) × float32 = 0.048 GB
      Outputs: (16694, 3) × float32 = 0.000 GB
      Total:   0.048 GB

  Actual memory allocated: 0.051 GB
  Loaded 16,694 samples into memory
  Effective compression ratio: 0.95x




wandb: Currently logged in as: andreas-hovaldt (be-ics-acs) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


In [3]:
trainer.train()

  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: Expected `device_type` of type `str`, got: `<class 'torch.device'>`

In [ ]:
len(trainer.trainloader)

33